In [1]:
%%capture
!pip install duckdb
!pip install ipython-sql
!pip install SQLAlchemy
!pip install duckdb-engine

#Drive mount

In [3]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


#Import & setup

In [4]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
from datetime import datetime as dt
import glob

# Import ipython-sql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: UserWarning: Config option `displaycon` not recognized by `SqlMagic`.  Did you mean `displaylimit`?
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
# %sql duckdb:////content/drive/MyDrive/ADSDBProject/formatted/crimesPrices.db
# to use a database file (shared between processes)
mostRecent = glob.glob("/content/drive/MyDrive/ADSDB/ADSDB Project/trusted/*.db")[-1]
con = duckdb.connect(database=mostRecent)

In [118]:
crimes = con.execute("SELECT * FROM crimes").df()
prices = con.execute("SELECT * FROM prices").df()

In [119]:
crimes['LSOA name'].isnull().sum()

1

In [120]:
crimes.dtypes

Crime ID                  object
Month                     object
Reported by               object
Falls within              object
Longitude                float64
Latitude                 float64
Location                  object
LSOA code                 object
LSOA name                 object
Crime type                object
Last outcome category     object
Context                  float64
dtype: object

Imputation of Na by KNN-method. We use only numeric (in this case float64) columns

We only use these two numerical columns to impute Na

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

Row with nan

In [121]:
datos = crimes[['Longitude','Latitude','LSOA name']]
a = datos.isnull().sum(axis=1)
print(a[a==1])
nan_row = datos.iloc[19454,:]

19454    1
dtype: int64


Data without nan

In [122]:
non_nan = datos.dropna()

In [62]:
X = non_nan[['Longitude','Latitude']].values
y = non_nan['LSOA name'].values
 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [63]:
n_neighbors = 7
 
knn = KNeighborsClassifier(n_neighbors)
knn.fit(X_train, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(X_test, y_test)))

Accuracy of K-NN classifier on training set: 0.98
Accuracy of K-NN classifier on test set: 0.98


In [64]:
pred = knn.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

[[ 9  0  0 ...  0  0  0]
 [ 0 27  0 ...  0  0  0]
 [ 0  0 10 ...  0  0  0]
 ...
 [ 0  0  0 ... 20  0  0]
 [ 0  0  0 ...  0 13  0]
 [ 0  0  0 ...  0  0 29]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                          precision    recall  f1-score   support

             Boston 001A       1.00      1.00      1.00         9
             Boston 001B       0.96      1.00      0.98        27
             Boston 001C       1.00      1.00      1.00        10
             Boston 001D       0.90      1.00      0.95        19
             Boston 001E       1.00      0.96      0.98        55
             Boston 002A       1.00      1.00      1.00        63
             Boston 002B       1.00      0.99      0.99        83
             Boston 002C       1.00      1.00      1.00        88
             Boston 002D       1.00      1.00      1.00       102
             Boston 002E       1.00      1.00      1.00        62
             Boston 003A       0.99      0.99      0.99       256
             Boston 003B       0.98      1.00      0.99        40
             Boston 003C       0.98      0.99      0.99       186
             Boston 003D       1.00      1.00      1.00       215
         

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [123]:
nan_row = nan_row[['Longitude', 'Latitude']]
nan_row
nan_row = nan_row.to_numpy()
nan_row = nan_row.reshape(1,2)

Imputation by knn

In [124]:
imp = knn.predict(nan_row)
imp

array(['West Lindsey 001G'], dtype=object)

In [126]:
crimes.iloc[19454,8] = 'West Lindsey 001G'